# Bird Detection in High Resolution Airborne Imagery
Credit to Ben Weinstein of Weecology at University of Florida for creating the Bird Detector model and the original implementation notebook (https://colab.research.google.com/drive/1e9_pZM0n_v3MkZpSjVRjm55-LuCE2IYE?usp=sharing). I have added a few additional comments for clarity.

In this exercise, we will load a machine learning model called Deepforest Bird Detector trained to identify birds in aerial imagery, and visualize the model's predictions of bird locations on the imagery.

# Install and load packages
These are all the packages that you need to run the bird detection model and visualize the model's predictions

In [1]:
pip install --upgrade deepforest albumentations PyYAML

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

## Load modules
Deepforest is a module for accessing a machine learning model that can be used to identify trees or birds from aerial imagery.

Rasterio is a module for displaying and working with raster data

Matplotlib is a module for making visualizations such as graphs and maps

Numpy is a module for working with mathematical arrays

In [2]:
from deepforest import main
import rasterio
from matplotlib import pyplot
import numpy as np

# Load Model

*   In this step, we are creating a variable to call the Deepforest model
*   By default, Deepforest is a tree identification model, so we have to specify to use the bird model instead



In [3]:
m = main.deepforest()
m.use_bird_release()

Reading config file: /usr/local/lib/python3.10/dist-packages/deepforest/data/deepforest_config.yml


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:03<00:00, 34.3MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


No validation file provided. Turning off validation loop


bird.pt: 129MB [00:01, 115MB/s]                           


Model was downloaded and saved to /usr/local/lib/python3.10/dist-packages/deepforest/data/bird.pt
Loading pre-built model: https://github.com/weecology/BirdDetector/releases/tag/1.1
Setting default score threshold to 0.3


# Download pelican image
This is a sample aerial image of a pelican rookery (nesting site) in Canada that we will try detecting birds on!

In [4]:
!wget https://www.dropbox.com/s/y6tt566khmt1ecn/AWPE%20Pigeon%20Lake%202020%20DJI_0005_crop.JPG?dl=0 -O pelicans.jpg

--2024-08-15 16:56:00--  https://www.dropbox.com/s/y6tt566khmt1ecn/AWPE%20Pigeon%20Lake%202020%20DJI_0005_crop.JPG?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/kthvou2ylr19kbcb1b6af/AWPE-Pigeon-Lake-2020-DJI_0005_crop.JPG?rlkey=dnbr5g5thqu1rn00zel2gw6np&dl=0 [following]
--2024-08-15 16:56:00--  https://www.dropbox.com/scl/fi/kthvou2ylr19kbcb1b6af/AWPE-Pigeon-Lake-2020-DJI_0005_crop.JPG?rlkey=dnbr5g5thqu1rn00zel2gw6np&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb7ce1b6532c31ffe2222b6d2cd.dl.dropboxusercontent.com/cd/0/inline/CYsdFWxD7Y_SzQzWblSFg6TjZrLWEW0Pu2rLrD2vRKVJXxoQrvGxooYqdiDj1yapNPmyBDmxJhboKlxvUEoGppS3HAR8HSGglrOYeBX3VMbebceNZxT76q2aD8Xv2kcI3g8/file# [following]
--2024-08-15 16:56

# Predict bird locations and plot results
Here, we call the model and ask it to make predictions of bird locations on the pelican image.

Then, we use matplotlib to display the model's predictions on the imagery as orange boxes around the objects it thinks are birds


In [5]:
plot = m.predict_image(path="pelicans.jpg",return_plot=True)

In [ ]:
pyplot.figure(figsize=(30,30))
#matplotlib likes RGB colors, but DeepForest-pytorch depends on openCV which like BGR colors. To plot, invert the color order.
pyplot.imshow(plot[:,:,::-1])

# Download penguins orthomosaic
This is to demonstrate that the model can make predictions on georeferenced imagery. In this case, we are calling up an orthomosaic of a Chinstrap Penguin colony in Antarctica

After generating the visualization of the orthomosaic using rasterio and matplotlib, zoom in on different parts of the island to get a better look at the colony!

In [6]:
!wget https://www.dropbox.com/s/yi9m8xvju2j99jq/Cabo_Cariz_2017_Chinstrap_penguins.tif?dl=0 -O "penguins.tif"

--2024-08-15 16:56:20--  https://www.dropbox.com/s/yi9m8xvju2j99jq/Cabo_Cariz_2017_Chinstrap_penguins.tif?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/b8qlfr7hnei243ncnm6s1/Cabo_Cariz_2017_Chinstrap_penguins.tif?rlkey=ke640sgq44cbfqw5kkgv02ffr&dl=0 [following]
--2024-08-15 16:56:20--  https://www.dropbox.com/scl/fi/b8qlfr7hnei243ncnm6s1/Cabo_Cariz_2017_Chinstrap_penguins.tif?rlkey=ke640sgq44cbfqw5kkgv02ffr&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9fea8bc207bbac6c086ca14c58.dl.dropboxusercontent.com/cd/0/inline/CYtHsD9e6KbvXya_7t9yw-FU_gPchG4g3J4cz0zSFsARqsvy9FbLfg_SW8YzzyQoUlOdmal-9Uo6NkCcANlW8BFkoQWe1SqMmLhWE4Czg8g7NNaMHJVs2Rz-3RMA-n3B3Ns/file# [following]
--2024-08-15 16:56:21--  http

In [7]:
raster = rasterio.open("penguins.tif").read()
#change from channels first (C, H, W) to channels last (H, W, C) for plotting
raster = np.rollaxis(raster,0,3)

In [10]:
pyplot.figure(figsize=(30,30))
pyplot.imshow(raster)

Output hidden; open in https://colab.research.google.com to view.

# Predict bird locations and plot results
As above, we ask the model to examine the orthomosaic and predict where birds are located on it. Then, we visualize the orthomosaic, with orange boxes representing the objects the model thinks are birds.

In [11]:
predicted_image =  m.predict_tile(raster_path= "penguins.tif", patch_size = 1000, return_plot=True)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
pyplot.figure(figsize=(30,30))
pyplot.imshow(predicted_image[:,:,::-1])

# Download Bosque del Apache image
Now we are going to test the model's performance on a drone image that the model wasn't trained on! While Deepforest Bird Detector was trained on thousands of bird images from around the world, it was not trained on any images that we collected at Bosque del Apache with our Wingtra One Gen II drone.

In [13]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1DkLbi_3MFJsvRAI4Vaomxibyv2ccfMSl' -O "bda.jpg"


--2024-08-15 17:08:24--  https://drive.google.com/uc?export=download&id=1DkLbi_3MFJsvRAI4Vaomxibyv2ccfMSl
Resolving drive.google.com (drive.google.com)... 142.250.125.139, 142.250.125.113, 142.250.125.138, ...
Connecting to drive.google.com (drive.google.com)|142.250.125.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1DkLbi_3MFJsvRAI4Vaomxibyv2ccfMSl&export=download [following]
--2024-08-15 17:08:24--  https://drive.usercontent.google.com/download?id=1DkLbi_3MFJsvRAI4Vaomxibyv2ccfMSl&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.125.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18161794 (17M) [image/jpeg]
Saving to: ‘bda.jpg’

bda.jpg             100%[===================>]  17.32M  56.2MB/s    in 0.3s    

# Predict bird locations and plot results

In [14]:
birds = m.predict_image(path="bda.jpg",return_plot=True)

In [15]:
pyplot.figure(figsize=(30,30))
#matplotlib likes RGB colors, but DeepForest-pytorch depends on openCV which like BGR colors. To plot, invert the color order.
pyplot.imshow(birds[:,:,::-1])

Output hidden; open in https://colab.research.google.com to view.